In [2]:
#su colab
#from google.colab import drive
#drive.mount('/content/drive')

#video_path = '/content/drive/MyDrive/mucche/mucchine.mp4'


In [3]:
#su vs code

video_path = 'DJI_0429_cut.mp4'

In [4]:
import cv2
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
        raise RuntimeError("Impossibile aprire il video.")


In [6]:
fps = 30

In [9]:

frame_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("frames:", frame_count)
duration = frame_count / fps
print("durata video:", duration, "s")

frames: 282
durata video: 9.4 s


In [10]:
!pip install ultralytics

In [11]:
import numpy as np
from collections import deque
import numpy as np
from ultralytics import YOLO

*Classe traker che ho copiato e adattato*:

Tracker called for each new frame with a list of detections (from a YOLO model).

Matching: It calculates the distances between the centroids of existing tracks and new detections. It then tries to match new detections to existing tracks based on proximity (greedy matching).

Updating: If an existing track is successfully matched to a new detection, its centroid and last_frame are updated.

New Tracks: Any detections that couldn't be matched to an existing track are considered new objects, and a new track with a unique ID is created for them.

Handling Lost Tracks: If a track is not matched in a given frame, its 'lost' counter increases. If this counter exceeds max_lost_frames, the track is removed, assuming the object has left the scene or is no longer detectable.

In essence, it takes raw object detections from each frame and tries to maintain a consistent identity for each object over time

In [13]:
class tracker:
    def __init__(self, max_lost_frames=5, max_distance=80):
        self.next_id = 0
        self.tracks = {}  # id -> dict{centroid, last_frame, history(deque of (frame_idx, centroid, speed, accel, grouping))}
        self.max_lost = max_lost_frames
        self.max_dist = max_distance

    def update(self, detections, frame_idx, timestamp):
        """
        detections: list of (cx, cy, bbox, conf)
        returns: list of (id, cx, cy, bbox, conf)
        """
        assigned = {}
        out = []

        # prepare arrays
        det_centroids = np.array([[d[0], d[1]] for d in detections]) if detections else np.empty((0, 2))
        track_ids = list(self.tracks.keys())
        track_centroids = np.array([self.tracks[t]['centroid'] for t in track_ids]) if track_ids else np.empty((0, 2))

        if len(track_centroids) and len(det_centroids):
            # distance matrix
            dmat = np.linalg.norm(track_centroids[:, None, :] - det_centroids[None, :, :], axis=2)
            # greedy matching
            while True:
                i, j = np.unravel_index(np.argmin(dmat), dmat.shape)
                if not np.isfinite(dmat[i, j]):
                    break
                if dmat[i, j] > self.max_dist:
                    break
                tid = track_ids[i]
                assigned[tid] = j
                dmat[i, :] = np.inf
                dmat[:, j] = np.inf
                if np.all(np.isinf(dmat)):
                    break

        used_dets = set(assigned.values())
        # update assigned tracks
        for tid, j in assigned.items():
            cx, cy, bbox, conf = detections[j]
            self.tracks[tid]['centroid'] = (cx, cy)
            self.tracks[tid]['last_frame'] = frame_idx
            out.append((tid, cx, cy, bbox, conf))

        # create new tracks for unassigned detections
        for idx, det in enumerate(detections):
            if idx in used_dets:
                continue
            cx, cy, bbox, conf = det
            tid = self.next_id
            self.next_id += 1
            self.tracks[tid] = {
                'centroid': (cx, cy),
                'last_frame': frame_idx,
                'history': deque(maxlen=256),  # keep history for speed/accel windows
                'lost': 0
            }
            out.append((tid, cx, cy, bbox, conf))

        # increment lost counters and remove stale tracks
        to_delete = []
        for tid in list(self.tracks.keys()):
            if self.tracks[tid]['last_frame'] != frame_idx:
                self.tracks[tid]['lost'] = self.tracks[tid].get('lost', 0) + 1
                if self.tracks[tid]['lost'] > self.max_lost:
                    to_delete.append(tid)
        for tid in to_delete:
            del self.tracks[tid]

        return out


In [14]:
trk = tracker(max_lost_frames=5, max_distance=120)

In [15]:
#per riconoscimento mucche uso yolo (classe 19)
model = YOLO('yolov8n.pt')

In [12]:
def id_to_color(track_id):
    np.random.seed(track_id)
    return tuple(int(c) for c in np.random.randint(0, 255, size=3))


# Prova detection

In [13]:
#BLOCCO CHE SALVA IL VIDEO CON IL TRAKING DELLE MUCCHE, metti true se lo vuoi salvare
debug_id = True
output_video = video_path.rsplit('.', 1)[0] + "_debug_id.mp4"
print(output_video)
if debug_id and output_video is not None:
    if output_video.endswith('.mp4'):
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter(output_video, fourcc, fps, (frame_w, frame_h))
    else:
        print("metti un mp4")
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        #results = model(frame, conf=0.4, iou=0.5)[0]
        results = model.predict(frame, imgsz=max(frame_w, frame_h), conf=0.2, verbose=False)[0]

        detections = []
        for box in results.boxes:
            cls = int(box.cls[0])
            if cls != 19:  # cow
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)
            conf = float(box.conf[0])

            detections.append((cx, cy, (x1, y1, x2, y2), conf))

        tracks = trk.update(detections, frame_idx, timestamp=None)

        for tid, cx, cy, bbox, conf in tracks:
            x1, y1, x2, y2 = bbox
            color = id_to_color(tid)

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(
                frame,
                f"ID {tid}",
                (x1, y1 - 5),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                color,
                2
            )

        writer.write(frame)
        frame_idx += 1

    cap.release()
    writer.release()
    print("Video mucche con id salvato.")


DJI_0429_cut_debug_id.mp4
Video mucche con id salvato.


# Prova direzioni

In [18]:
def estimate_camera_motion_farneback(prev_gray, gray, centroids):
    """
    Stima il movimento della camera usando Farneback optical flow denso,
    escludendo le zone intorno ai centroidi delle mucche.
    centroids: lista di (cx, cy) da mascherare
    """
    if prev_gray is None or gray is None:
        return 0.0, 0.0

    #crea maschera escludendo cerchi intorno ai centroidi delle mucche

    mask = np.ones_like(gray, dtype=np.uint8) * 255
    if centroids is not None:
        for (cx, cy) in centroids:
            x, y = int(cx), int(cy)
            cv2.circle(mask, (x, y), 30, 0, -1)

    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None,
                                        0.5, 3, 15, 3, 5, 1.2, 0)

    valid_mask = mask > 0
    if not np.any(valid_mask):
        return 0.0, 0.0

    dx = float(np.median(flow[:, :, 0][valid_mask]))
    dy = float(np.median(flow[:, :, 1][valid_mask]))

    return dx, dy


In [15]:
debug_frecce = True
output_video = video_path.rsplit('.', 1)[0] + "_debug_frecce.mp4"
print(output_video)
if debug_frecce and output_video is not None:
    if output_video.endswith('.mp4'):
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter(output_video, fourcc, fps, (frame_w, frame_h))
    else:
        print("metti un mp4")


    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
            raise RuntimeError("Impossibile aprire il video.")
        

    frame_idx = 0
    cam_dx, cam_dy = 0.0, 0.0
    prev_gray = None
    camera_motion_history = deque(maxlen=10)  # storia ultimi 10 frame per smoothing
    COW_MOTION_THRESHOLD = 4  # px/frame (regolabile)

    # Dizionario per tracciare posizioni precedenti delle mucche
    prev_cow_positions = {}
    cow_disturbance_index = {}  # indice di disturbo per ogni mucca
    cow_movement_history = {}  # storia movimenti per ogni mucca per smoothing

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        #results = model(frame, conf=0.4, iou=0.5)[0]
        results = model.predict(frame, imgsz=max(frame_w, frame_h), conf=0.2, verbose=False)[0]
        detections = []
        for box in results.boxes:
            cls = int(box.cls[0])
            if cls != 19:  # cow
                continue
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)
            conf = float(box.conf[0])
            detections.append((cx, cy, (x1, y1, x2, y2), conf))
        tracks = trk.update(detections, frame_idx, timestamp=None)
        #bboxes = [bbox for tid, cx, cy, bbox, conf in tracks]
        centroids = {tid: (cx, cy) for (tid, cx, cy, _, _) in tracks}
        centroid_list = list(centroids.values())

        if prev_gray is not None:
            cam_dx_raw, cam_dy_raw = estimate_camera_motion_farneback(prev_gray, gray, centroid_list)
            camera_motion_history.append((cam_dx_raw, cam_dy_raw))
            #media sugli ultimi N frame per smoothing (per non far vibrare troppo le frecce)
            if len(camera_motion_history) > 0:
                cam_dx = float(np.mean([m[0] for m in camera_motion_history]))
                cam_dy = float(np.mean([m[1] for m in camera_motion_history]))
            else:
                cam_dx, cam_dy = cam_dx_raw, cam_dy_raw
        else:
            cam_dx, cam_dy = 0.0, 0.0
            
        
        #calcola movimento mucca
        cow_movements = {}
        for tid, cx, cy, bbox, conf in tracks:
            if tid in prev_cow_positions:
                prev_cx, prev_cy = prev_cow_positions[tid]
                # Movimento assoluto della mucca
                abs_dx = cx - prev_cx
                abs_dy = cy - prev_cy
                # Movimento relativo (compensando il movimento della camera)
                rel_dx = abs_dx - cam_dx
                rel_dy = abs_dy - cam_dy
                # Magnitudine del movimento
                movement_magnitude = np.sqrt(rel_dx**2 + rel_dy**2)
                
                # Aggiorna storia movimenti per smoothing
                if tid not in cow_movement_history:
                    cow_movement_history[tid] = deque(maxlen=5)  # ultimi 5 frame per smoothing
                cow_movement_history[tid].append((rel_dx, rel_dy, movement_magnitude))
                
                # Calcola movimento smoothed
                if len(cow_movement_history[tid]) > 0:
                    smooth_dx = float(np.mean([m[0] for m in cow_movement_history[tid]]))
                    smooth_dy = float(np.mean([m[1] for m in cow_movement_history[tid]]))
                    smooth_magnitude = float(np.mean([m[2] for m in cow_movement_history[tid]]))
                else:
                    smooth_dx, smooth_dy, smooth_magnitude = rel_dx, rel_dy, movement_magnitude
                
                cow_movements[tid] = (smooth_dx, smooth_dy, smooth_magnitude)
                
                # Aggiorna indice di disturbo (media mobile del movimento)
                if tid not in cow_disturbance_index:
                    cow_disturbance_index[tid] = deque(maxlen=30)  # ultimi 30 frame
                cow_disturbance_index[tid].append(movement_magnitude)
            else:
                cow_movements[tid] = (0.0, 0.0, 0.0)
            
            # Aggiorna posizione precedente
            prev_cow_positions[tid] = (cx, cy)


        if debug_frecce:
            #disegna freccia camera in alto a destra (invertita per mostrare direzione movimento camera)
            cam_arrow_start = (frame_w - 50, 50)
            cam_arrow_end = (int(frame_w - 50 - cam_dx * 10), int(50 - cam_dy * 10))  # scala x10, invertita
            cv2.arrowedLine(frame, cam_arrow_start, cam_arrow_end, (0, 255, 255), 3, tipLength=0.3)
            cv2.putText(frame, "Camera", (frame_w - 120, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            
            #disegna frecce mucche solo se si muovono più di una soglia COW_MOTION_THRESHOLD
            for tid, cx, cy, bbox, conf in tracks:
                color = id_to_color(tid)
                x1, y1, x2, y2 = bbox
                
                # Disegna bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                
                if tid in cow_movements:
                    rel_dx, rel_dy, movement_magnitude = cow_movements[tid]
                    
                    # Disegna freccia solo se il movimento supera la soglia
                    if movement_magnitude > COW_MOTION_THRESHOLD:
                        arrow_start = (cx, cy)
                        arrow_end = (int(cx + rel_dx * 5), int(cy + rel_dy * 5))  # scala x5 per visibilità
                        cv2.arrowedLine(frame, arrow_start, arrow_end, color, 2, tipLength=0.3)
            
            #scrivi indice di disturbo per ogni mucca
            for tid, cx, cy, bbox, conf in tracks:
                x1, y1, x2, y2 = bbox
                color = id_to_color(tid)
                
                # Calcola indice di disturbo medio
                if tid in cow_disturbance_index and len(cow_disturbance_index[tid]) > 0:
                    avg_disturbance = np.mean(cow_disturbance_index[tid])
                    disturbance_text = f"ID {tid} - CDI: {avg_disturbance:.2f}"
                else:
                    disturbance_text = f"ID {tid} - CDI: 0.00"
                
                cv2.putText(frame, disturbance_text, (x1, y1 - 5), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)



        writer.write(frame)
        frame_idx += 1
        prev_gray = gray.copy()
        
    cap.release()
    writer.release()
    print("Video mucche con id salvato.")

DJI_0429_cut_debug_frecce.mp4
Video mucche con id salvato.


# Prova indice di disturbo basato su direzioni e velocità

In [19]:
output_video = video_path.rsplit('.', 1)[0] + "_cdi_visual.mp4"
print(output_video)
if output_video.endswith('.mp4'):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(output_video, fourcc, fps, (frame_w, frame_h))
else:
    print("metti un mp4")
debug_frecce = False

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise RuntimeError("Impossibile aprire il video.")
    

frame_idx = 0
cam_dx, cam_dy = 0.0, 0.0
prev_gray = None
camera_motion_history = deque(maxlen=10)  # storia ultimi 10 frame per smoothing
COW_MOTION_THRESHOLD = 4  # px/frame (regolabile)
CDI_MAX = 10.0  # valore massimo per normalizzare l'indice di disturbo (regolabile)

# Dizionario per tracciare posizioni precedenti delle mucche
prev_cow_positions = {}
cow_disturbance_index = {}  # indice di disturbo per ogni mucca
cow_movement_history = {}  # storia movimenti per ogni mucca per smoothing

def get_cdi_color(cdi_value, max_cdi=10.0):
    """
    Restituisce un colore BGR in base al CDI (verde -> giallo -> rosso)
    cdi_value: valore CDI da 0 a max_cdi
    """
    # Normalizza tra 0 e 1
    normalized = min(cdi_value / max_cdi, 1.0)
    
    if normalized < 0.5:
        # Verde -> Giallo (0-0.5)
        # B rimane 0, G rimane 255, R aumenta
        r = int(normalized * 2 * 255)
        g = 255
        b = 0
    else:
        # Giallo -> Rosso (0.5-1.0)
        # B rimane 0, R rimane 255, G diminuisce
        r = 255
        g = int((1.0 - normalized) * 2 * 255)
        b = 0
    
    return (b, g, r)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    results = model.predict(frame, imgsz=max(frame_w, frame_h), conf=0.2, verbose=False)[0]
    detections = []
    for box in results.boxes:
        cls = int(box.cls[0])
        if cls != 19:  # cow
            continue
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cx = int((x1 + x2) / 2)
        cy = int((y1 + y2) / 2)
        conf = float(box.conf[0])
        detections.append((cx, cy, (x1, y1, x2, y2), conf))
    tracks = trk.update(detections, frame_idx, timestamp=None)
    centroids = {tid: (cx, cy) for (tid, cx, cy, _, _) in tracks}
    centroid_list = list(centroids.values())

    if prev_gray is not None:
        cam_dx_raw, cam_dy_raw = estimate_camera_motion_farneback(prev_gray, gray, centroid_list)
        camera_motion_history.append((cam_dx_raw, cam_dy_raw))
        if len(camera_motion_history) > 0:
            cam_dx = float(np.mean([m[0] for m in camera_motion_history]))
            cam_dy = float(np.mean([m[1] for m in camera_motion_history]))
        else:
            cam_dx, cam_dy = cam_dx_raw, cam_dy_raw
    else:
        cam_dx, cam_dy = 0.0, 0.0
        
    
    # Calcola movimento mucca
    cow_movements = {}
    for tid, cx, cy, bbox, conf in tracks:
        if tid in prev_cow_positions:
            prev_cx, prev_cy = prev_cow_positions[tid]
            abs_dx = cx - prev_cx
            abs_dy = cy - prev_cy
            rel_dx = abs_dx - cam_dx
            rel_dy = abs_dy - cam_dy
            movement_magnitude = np.sqrt(rel_dx**2 + rel_dy**2)
            
            if tid not in cow_movement_history:
                cow_movement_history[tid] = deque(maxlen=5)
            cow_movement_history[tid].append((rel_dx, rel_dy, movement_magnitude))
            
            if len(cow_movement_history[tid]) > 0:
                smooth_dx = float(np.mean([m[0] for m in cow_movement_history[tid]]))
                smooth_dy = float(np.mean([m[1] for m in cow_movement_history[tid]]))
                smooth_magnitude = float(np.mean([m[2] for m in cow_movement_history[tid]]))
            else:
                smooth_dx, smooth_dy, smooth_magnitude = rel_dx, rel_dy, movement_magnitude
            
            cow_movements[tid] = (smooth_dx, smooth_dy, smooth_magnitude)
            
            if tid not in cow_disturbance_index:
                cow_disturbance_index[tid] = deque(maxlen=30)
            cow_disturbance_index[tid].append(movement_magnitude)
        else:
            cow_movements[tid] = (0.0, 0.0, 0.0)
        
        prev_cow_positions[tid] = (cx, cy)

    # Disegna indicatori CDI
    cam_arrow_start = (frame_w - 50, 50)
    cam_arrow_end = (int(frame_w - 50 - cam_dx * 10), int(50 - cam_dy * 10))
    cv2.arrowedLine(frame, cam_arrow_start, cam_arrow_end, (0, 255, 255), 2, tipLength=0.3)
    cv2.putText(frame, "Cam", (frame_w - 90, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
    
    for tid, cx, cy, bbox, conf in tracks:
        x1, y1, x2, y2 = bbox
        
        # Calcola CDI medio
        if tid in cow_disturbance_index and len(cow_disturbance_index[tid]) > 0:
            avg_disturbance = np.mean(cow_disturbance_index[tid])
        else:
            avg_disturbance = 0.0
        
        # Ottieni colore in base al CDI
        color = get_cdi_color(avg_disturbance, CDI_MAX)
        
        # Disegna bounding box con colore CDI
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
        
        # Disegna barra CDI sopra la mucca
        bar_width = x2 - x1
        bar_height = 10
        bar_x1 = x1
        bar_y1 = y1 - 25
        bar_x2 = x2
        bar_y2 = y1 - 15
        
        # Sfondo barra (grigio)
        cv2.rectangle(frame, (bar_x1, bar_y1), (bar_x2, bar_y2), (80, 80, 80), -1)
        
        # Riempimento barra in base al CDI
        fill_width = int(bar_width * min(avg_disturbance / CDI_MAX, 1.0))
        if fill_width > 0:
            cv2.rectangle(frame, (bar_x1, bar_y1), (bar_x1 + fill_width, bar_y2), color, -1)
        
        # Bordo barra
        cv2.rectangle(frame, (bar_x1, bar_y1), (bar_x2, bar_y2), (255, 255, 255), 1)
        
        # Testo CDI
        disturbance_text = f"ID{tid}: {avg_disturbance:.1f}"
        cv2.putText(frame, disturbance_text, (x1, y1 - 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.putText(frame, disturbance_text, (x1, y1 - 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        # Disegna freccia movimento se supera soglia (opzionale)
        if debug_frecce and tid in cow_movements:
            rel_dx, rel_dy, movement_magnitude = cow_movements[tid]
            if movement_magnitude > COW_MOTION_THRESHOLD:
                arrow_start = (cx, cy)
                arrow_end = (int(cx + rel_dx * 5), int(cy + rel_dy * 5))
                cv2.arrowedLine(frame, arrow_start, arrow_end, color, 2, tipLength=0.3)

    writer.write(frame)
    frame_idx += 1
    prev_gray = gray.copy()
    
cap.release()
writer.release()
print("Video con indice di disturbo visivo salvato.")

DJI_0429_cut_cdi_visual.mp4
Video con indice di disturbo visivo salvato.


To do:

- ~~prova occhi~~
- indice medio
- grafico disturbo nel tempo
- prova su video esteso